In [6]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df =pd.read_csv('../data/student-por.csv')


In [12]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  paid        649 non-null    object
 18  activities  649 non-null    object
 19  nursery     649 non-null    object
 20  higher    

In [14]:
df.isnull().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [16]:
df.duplicated().sum()

0

In [17]:
# Data Type Conversion 
# Convert binary 'yes'/'no' columns to a more efficient boolean type (True/False).

binary_cols = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
    
for col in binary_cols:
    df[col] = df[col].map({'yes': True, 'no': False})
    print(f" - Converted '{col}' column.")


 - Converted 'schoolsup' column.
 - Converted 'famsup' column.
 - Converted 'paid' column.
 - Converted 'activities' column.
 - Converted 'nursery' column.
 - Converted 'higher' column.
 - Converted 'internet' column.
 - Converted 'romantic' column.


In [18]:
# Feature Engineering 

# Create a 'total alcohol consumption' column
df['total_alcohol'] = (2 * df['Dalc'] + 5 * df['Walc']) / 7

In [19]:
# Create a 'parental_education' score.
df['parental_education'] = df['Medu'] + df['Fedu']

In [33]:
# Define a function for Academic Trajectory
# This function categorizes students based on their grades (G1, G2, G3).
# This is a much more nuanced approach than a simple pass/fail status.

def get_academic_trajectory(row):
    passing_grade = 10
    g1_pass = row['G1'] >= passing_grade
    g2_pass = row['G2'] >= passing_grade
    g3_pass = row['G3'] >= passing_grade
    
    # Consistently high-performing students
    if g1_pass and g2_pass and g3_pass:
        return 'Consistent Performer'
    # Students who are struggling the most
    elif not g1_pass and not g2_pass and not g3_pass:
        return 'Struggling'
    # Students showing a clear downward trend
    elif g1_pass and not g3_pass:
        return 'Declining'
    # Students showing clear improvement
    elif not g1_pass and g3_pass:
        return 'Improving'
    # All other mixed cases
    else:
        return 'Inconsistent'


In [34]:
# Apply the new academic trajectory function to create our target column.
df['academic_trajectory'] = df.apply(get_academic_trajectory, axis=1)


In [35]:
df['academic_trajectory']

0                 Improving
1                 Improving
2      Consistent Performer
3      Consistent Performer
4      Consistent Performer
               ...         
644    Consistent Performer
645    Consistent Performer
646               Declining
647    Consistent Performer
648    Consistent Performer
Name: academic_trajectory, Length: 649, dtype: object

In [36]:
df.to_csv('../data/cleaned_student_data.csv', index=False)